In [1]:
import time

import google.generativeai as genai
import pandas as pd
from tqdm import tqdm

In [2]:
# https://ai.google.dev/gemini-api/docs/api-key
api_key = '<YOUR_API_KEY'
genai.configure(api_key=api_key)

df = pd.read_csv('LK_modified.xlsx - Вопрос ответ.csv')
df = df[['question', 'content', 'category']]
df = df[~df['content'].duplicated(keep=False)]

model = genai.GenerativeModel('gemini-1.5-pro')


def get_synthetic_questions(model: genai.GenerativeModel, df: pd.DataFrame):
    synthetic_data = []

    for question, answer, category in tqdm(df[['question', 'content', 'category']].head(n=5).values):
        request = f"""
            Вот несколько примеров вопросов, которые обычно задают во внутреннюю текстовую поддержку, и заранее подготовленных типовых ответов на них:

            1. Вопрос: больничный по уходу за родственником
            Ответ (фиксированный): Больничный с кодом 09 (уход за больным членом семьи) оплачивается полностью за счет средств СФР, работодатель в данном случае выплату не производит.

            2. Вопрос: эцп не активка
            Ответ (фиксированный): Выпуск УНЭП ЮЛ осуществляется в Личный кабинете - Блок «Электронные подписи». Данный блок можно найти в правом верхнем углу, в выпадающем списке под своими ФИО. На странице Вы найдете соответствующую плашку. На данной плашке необходимо нажать «Получить электронную подпись». Подробная инструкция доступна по ссылке

            3. Вопрос: мне нужно изменить % занятости
            Ответ (фиксированный): Для изменения процента занятости сотруднику создайте, пожалуйста, заявку на сотрудника по теме "Изменение режима, характера работы"


            Далее тебе нужно сгененировать 3 вопроса, релевантных приведенному ниже ответу. Обрати внимание, что сам ответ генерировать тебе не нужно, а вопросы могут иметь орфографические и пунктуационные ошибки, начинаться с маленькой буквы или иметь другие огрехи текстового ввода.

            Вопрос: {question}
            Ответ (фиксированный, его не нужно генерировать): {answer}

            Приведи в пример еще 3 вопроса, на которые был бы ответ выше, записав их в отдельные строки через символ \n без дополнительной информации, перечисления цифрами или использования черточек:
        """
        response = model.generate_content(request).text

        for example in response.split('\n'):
            if example:
                synthetic_data.append((question, example, category))

        time.sleep(30)

    return synthetic_data


synthetic_data = get_synthetic_questions(model, df)
synthetic_df = pd.DataFrame(synthetic_data, columns=['question', 'content', 'category'])

100%|██████████| 5/5 [02:42<00:00, 32.53s/it]


In [3]:
synthetic_df.to_csv('synthetic_df.csv', index=False)
synthetic_df.tail()

,question,content,category
10,Какие документы нужно предоставить для оформле...,какие документы нужны чтобы уйти в декрет,БиР
11,Какие документы нужно предоставить для оформле...,что нужно чтобы получить декретные,БиР
12,"Когда отпуск по БИР закончился, нужно ли писат...",хочу в отпуск по уходу за ребенком,БиР
13,"Когда отпуск по БИР закончился, нужно ли писат...",выплатили декретные что делать дальше,БиР
14,"Когда отпуск по БИР закончился, нужно ли писат...",отпуск по уходу за ребенком как оформить,БиР
